In [3]:
import sys
sys.path.append('../MGO')
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib as mpl
import sympy as sp
from IPython.display import display, Math

In [4]:
tau, t, E, k, x = sp.symbols('tau t E k x')
k0 = sp.symbols('k_0', positive=True)
D = - k**2 - x
RHS_x = - sp.diff(D, k)
RHS_k = sp.diff(D, x)
display(Math('D = '), D)

display('RHS_x', RHS_x)
display('RHS_k', RHS_k)

x, k = sp.Function('x'), sp.Function('k')
ode = [sp.Derivative(x(tau), tau) - RHS_x.subs({'k': k(tau), 'x': x(tau)}),
       sp.Derivative(k(tau), tau) - RHS_k.subs({'k': k(tau), 'x': x(tau)})]
ics = {k(0): k0, x(0): -k0**2}
sol = sp.dsolve(ode, ics=ics)
display(*sol)
x, k = sp.factor(sol[0].rhs), sp.factor(sol[1].rhs)
z = sp.Matrix([[x], [k]])
print('latex z: ', sp.latex(z))
display(Math('z ='), z)
display(Math('dz/d tau ='), sp.diff(z, tau))

Nt_int = sp.integrate(k*sp.diff(x, tau), (tau, 0, t))
display('Nt_int', Nt_int)
print('Nt_int', sp.latex(Nt_int))


<IPython.core.display.Math object>

-k**2 - x

'RHS_x'

2*k

'RHS_k'

...

<IPython.core.display.Math object>

Matrix([
[2*k_0 - 2*tau],
[           -1]])

'Nt_int'

2*k_0**2*t - 2*k_0*t**2 + 2*t**3/3

Nt_int 2 k_{0}^{2} t - 2 k_{0} t^{2} + \frac{2 t^{3}}{3}


**Manual Simplification**

Notice that $z = (-k^2(\tau), k(\tau))$ and $\frac{dz}{d\tau} = (2 k(\tau), -1)$.

In [5]:
k = sp.Function('k', real=True)
dzdtau = sp.Matrix([[2*k(tau)], [-1]])
display(dzdtau)
print('dzdtau:', sp.latex(dzdtau))

Q, R = dzdtau.QRdecomposition()
Q_t, R_t = Q.subs(tau, t), R[0].subs(tau, t)
A_t, B_t = sp.Matrix([Q_t[0]]), sp.Matrix([Q_t[1]])
Lambda_t = B_t.singular_values()[0]
display('Q_t:', Q_t)
display('R_t:', R_t)
display('A_t:', A_t)
display('B_t:', B_t)

print('Q_t:', sp.latex(Q_t))
print('R_t:', sp.latex(R_t))

sigma_t = 1
N_t = (sigma_t * sp.exp(sp.I * Nt_int) /
       (sp.sqrt(-2*sp.pi*sp.I) * sp.sqrt(Lambda_t * R_t)))
display('N_t:', N_t)

Matrix([
[2*k(tau)],
[      -1]])

dzdtau: \left[\begin{matrix}2 k{\left(\tau \right)}\\-1\end{matrix}\right]


'Q_t:'

Matrix([
[2*k(t)/sqrt(4*k(t)**2 + 1)],
[    -1/sqrt(4*k(t)**2 + 1)]])

'R_t:'

...

'B_t:'

Matrix([[-1/sqrt(4*k(t)**2 + 1)]])

Q_t: \left[\begin{matrix}\frac{2 k{\left(t \right)}}{\sqrt{4 k^{2}{\left(t \right)} + 1}}\\- \frac{1}{\sqrt{4 k^{2}{\left(t \right)} + 1}}\end{matrix}\right]
R_t: \sqrt{4 k^{2}{\left(t \right)} + 1}


'N_t:'

sqrt(2)*exp(I*(2*k_0**2*t - 2*k_0*t**2 + 2*t**3/3))/(2*sqrt(pi)*sqrt(-I))

**Manual simplifaction**

Let $\vartheta_t = \sqrt{4k^2(t) + 1}$

In [6]:
v_t = sp.symbols('vartheta_t')
z = sp.Matrix([-k(tau)**2, k(tau)])
A_t = 2*k(t)/v_t
B_t = -1/v_t
display('A_t', A_t, 'B_t', B_t)
S_t = sp.Matrix([[A_t, B_t], [-B_t, A_t]])
display('S_t', S_t)
Z_t = S_t@z
display('Z_t', Z_t)

'A_t'

2*k(t)/vartheta_t

'B_t'

-1/vartheta_t

'S_t'

Matrix([
[2*k(t)/vartheta_t,     -1/vartheta_t],
[     1/vartheta_t, 2*k(t)/vartheta_t]])

'Z_t'

Matrix([
[-2*k(t)*k(tau)**2/vartheta_t - k(tau)/vartheta_t],
[ 2*k(t)*k(tau)/vartheta_t - k(tau)**2/vartheta_t]])

In [7]:
X_t = Z_t[0]
gradtau_X_t = sp.simplify(sp.diff(X_t, tau).subs(sp.Derivative(k(tau), tau), -1))
display('gradtau_X_t', gradtau_X_t)
print('gradtau_X_t:', sp.latex(gradtau_X_t))

'gradtau_X_t'

(4*k(t)*k(tau) + 1)/vartheta_t

gradtau_X_t: \frac{4 k{\left(t \right)} k{\left(\tau \right)} + 1}{\vartheta_{t}}


In [8]:
J_t = gradtau_X_t
Phi_t = sp.sqrt(J_t.subs(tau, t)/J_t.subs(tau, tau))
display('Phi_t', Phi_t)

'Phi_t'

sqrt(4*k(t)**2 + 1)*sqrt(1/(4*k(t)*k(tau) + 1))

In [9]:
Phi_t = v_t / sp.sqrt(1 + 4 * k(t) * k(tau))
display('Phi_t', Phi_t)

'Phi_t'

vartheta_t/sqrt(4*k(t)*k(tau) + 1)

In [10]:
sp.simplify(J_t*Z_t[1]).subs(k(tau), k(t) + (t-tau))

(4*(t - tau + k(t))*k(t) + 1)*(-t + tau + k(t))*(t - tau + k(t))/vartheta_t**2

In [11]:
X_t = sp.symbols('X_t', real=True)
tau_X_t_sols = sp.solve(sp.Eq(X_t, sp.simplify(Z_t[0]).subs(k(tau), k0 - tau)), tau)
display('tau_X_t_sols', tau_X_t_sols[0], tau_X_t_sols[1])

'tau_X_t_sols'

(4*k_0*k(t) - sqrt(-8*X_t*vartheta_t*k(t) + 1) + 1)/(4*k(t))

(4*k_0*k(t) + sqrt(-8*X_t*vartheta_t*k(t) + 1) + 1)/(4*k(t))

In [12]:
tau_X_t = tau_X_t_sols[0]

In [16]:
X_t = sp.symbols('X_t', real=True)
k0_minus_tau_X_t = - (1 - sp.sqrt(1 - 8*k(t)*v_t*X_t))/(4*k(t))
Phi_t = v_t / sp.sqrt(1 + 4 * k(t) * k0_minus_tau_X_t)
Phi_t

vartheta_t/(-8*X_t*vartheta_t*k(t) + 1)**(1/4)

In [17]:
X_t = sp.symbols('X_t', real=True)
tau_X_t_sols = sp.solve(sp.Eq(X_t, sp.simplify(Z_t[0]).subs(k(tau), k0 - tau)), tau)
display('tau_X_t_sols', tau_X_t_sols[0], tau_X_t_sols[1])

'tau_X_t_sols'

(4*k_0*k(t) - sqrt(-8*X_t*vartheta_t*k(t) + 1) + 1)/(4*k(t))

(4*k_0*k(t) + sqrt(-8*X_t*vartheta_t*k(t) + 1) + 1)/(4*k(t))

**Manual Simplification**

Choose a branch and simplify

In [18]:
tau_X_t = k0 + (1 - sp.sqrt(1-8*X_t*v_t*k(t)))/(4*k(t))
display('tau_X_t', tau_X_t)
print('tau_X_t:', sp.latex(tau_X_t))

k_X_T = k0 - tau_X_t
display(k_X_T)

K_X_t = Z_t[1].subs(k(tau), k_X_T)
display('K_X_t', K_X_t)
display('K_X_t', sp.simplify(K_X_t.subs(v_t, sp.sqrt(1+4*k(t)**2))))

'tau_X_t'

k_0 + (1 - sqrt(-8*X_t*vartheta_t*k(t) + 1))/(4*k(t))

tau_X_t: k_{0} + \frac{1 - \sqrt{- 8 X_{t} \vartheta_{t} k{\left(t \right)} + 1}}{4 k{\left(t \right)}}


-(1 - sqrt(-8*X_t*vartheta_t*k(t) + 1))/(4*k(t))

'K_X_t'

-(1 - sqrt(-8*X_t*vartheta_t*k(t) + 1))**2/(16*vartheta_t*k(t)**2) - (1 - sqrt(-8*X_t*vartheta_t*k(t) + 1))/(2*vartheta_t)

'K_X_t'

(4*X_t*k(t) + sqrt(-8*X_t*sqrt(4*k(t)**2 + 1)*k(t) + 1)*sqrt(4*k(t)**2 + 1) - sqrt(4*k(t)**2 + 1))/(8*k(t)**2)

**Manual Simplification**

In [19]:
K_X_t = ((4*X_t*k(t)
          + v_t * (
              -1 + sp.sqrt(1 - 8 * k(t) * v_t * X_t)) )
         /( 8 * k(t)**2 ))
display('K_X_t', K_X_t)
display('K_X_t', sp.latex(K_X_t))

'K_X_t'

(4*X_t*k(t) + vartheta_t*(sqrt(-8*X_t*vartheta_t*k(t) + 1) - 1))/(8*k(t)**2)

'K_X_t'

'\\frac{4 X_{t} k{\\left(t \\right)} + \\vartheta_{t} \\left(\\sqrt{- 8 X_{t} \\vartheta_{t} k{\\left(t \\right)} + 1} - 1\\right)}{8 k^{2}{\\left(t \\right)}}'

In [20]:
epsilon = sp.symbols('epsilon')
sp.integrate(K_X_t, X_t, X_t)

(2*X_t**3*k(t)/3 - X_t**2*vartheta_t/2 + 2*vartheta_t*Piecewise((((-8*X_t*vartheta_t*k(t) + 1)**(5/2)/5 - (-8*X_t*vartheta_t*k(t) + 1)**(3/2)/3)/(32*vartheta_t**2*k(t)**2), Ne(vartheta_t*k(t), 0)), (X_t**2/2, True))/3 - Piecewise((-(-8*X_t*vartheta_t*k(t) + 1)**(3/2)/(12*vartheta_t*k(t)), Ne(vartheta_t*k(t), 0)), (X_t, True))/(12*k(t)))/(8*k(t)**2)

manual simplification

In [21]:
str(tau_X_t_sols[1])

'(4*k_0*k(t) + sqrt(-8*X_t*vartheta_t*k(t) + 1) + 1)/(4*k(t))'

**Manual Simplification**

Notice that $z = (-k^2(\tau), k(\tau))$ and $\frac{dz}{d\tau} = (2 k(\tau), -1)$.

In [25]:
k = sp.Function('k', real=True)
dzdtau = sp.Matrix([[2*k(tau)], [-1]])
display(dzdtau)
print('dzdtau:', sp.latex(dzdtau))

Q, R = dzdtau.QRdecomposition()
Q_t, R_t = Q.subs(tau, t), R[0].subs(tau, t)
A_t, B_t = sp.Matrix([Q_t[0]]), sp.Matrix([Q_t[1]])
Lambda_t = B_t.singular_values()[0]
display('Q_t:', Q_t)
display('R_t:', R_t)
display('A_t:', A_t)
display('B_t:', B_t)

print('Q_t:', sp.latex(Q_t))
print('R_t:', sp.latex(R_t))

sigma_t = 1
N_t = (sigma_t * sp.exp(sp.I * Nt_int) /
       (sp.sqrt(-2*sp.pi*sp.I) * sp.sqrt(Lambda_t * R_t)))
display('N_t:', N_t)

Matrix([
[2*k(tau)],
[      -1]])

dzdtau: \left[\begin{matrix}2 k{\left(\tau \right)}\\-1\end{matrix}\right]


'Q_t:'

Matrix([
[2*k(t)/sqrt(4*k(t)**2 + 1)],
[    -1/sqrt(4*k(t)**2 + 1)]])

'R_t:'

...

'B_t:'

Matrix([[-1/sqrt(4*k(t)**2 + 1)]])

Q_t: \left[\begin{matrix}\frac{2 k{\left(t \right)}}{\sqrt{4 k^{2}{\left(t \right)} + 1}}\\- \frac{1}{\sqrt{4 k^{2}{\left(t \right)} + 1}}\end{matrix}\right]
R_t: \sqrt{4 k^{2}{\left(t \right)} + 1}


'N_t:'

sqrt(2)*exp(I*(2*k_0**2*t - 2*k_0*t**2 + 2*t**3/3))/(2*sqrt(pi)*sqrt(-I))